# Visualizing a Simple Artificial Neural Network

_Sophie Libkind and Adam Shai_

_April 2017_

In this notebook we go through a simple artificial neural network (ANN) that will classify whether a point, $(x,y)$, comes from a circle of radius 1 or a circle of radius 2. We focus on the explicit visualization of the action of the ANN, in order to help build an intuition for how ANNs compute.

In [64]:
# Import necessary libraries for what's ahead

import math
import tensorflow as tf
from random import random, choice
import numpy
import numpy as np
import numpy.matlib
from matplotlib import cm

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, HoverTool
from bokeh.plotting import *

output_notebook()

#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook  

#from mpl_toolkits.mplot3d import Axes3D

Loading BokehJS ...

### The Training Set

In order to train our network we will need a training set. Here we define a function, _makeTraining_, that will return $n$ 2-D points, $(x,y)$, as well as the training label associated with each. The training labels take the form of a vector with one element set to 1, and all others set to 0, which is called a _one-hot vector_. The position of the 1 in the vector signifies the label. Here, a point coming from the circle of radius 1 is labeled $[1,0]$ whereas a point coming from the circle of radius 2 is labeled $[0,1]$.


In [7]:
def makeTraining(n = 1000):
    # n is the number of training examples you want outputed

    trainingData = [] # the list of (x,y) points
    trainingLabels = [] # the list of the labels for each (x,y) point in trainingData
    
    for i in range(n):
        
        # randomly choose a radius of 1 or 2
        radius = choice([1,2]) 
        
        # randomly choose an angle to sample
        theta = random() * 2 * math.pi
        
        # get the (x,y) point
        point = [radius * math.cos(theta), radius * math.sin(theta) ]
        
        # get the label
        if radius == 1:
            label = [1,0]
        else:
            label = [0,1]
        
        # append the (x,y) point and label to our lists
        trainingData.append(point)
        trainingLabels.append(label)               

    return trainingData, trainingLabels

Let's look at an example training set. Try hovering over points to see their $(x,y)$ values and their label.

In [73]:
# make an n=1000 training set
data_xy, labels_xy = makeTraining(100)

# make an interactive plot
source = ColumnDataSource(data=dict(x_data=[item[0] for item in data_xy],
                                    y_data=[item[1] for item in data_xy],
                                    labels=[str(item) for item in labels_xy]))

hover = HoverTool( tooltips=[ ("(x,y)", "(@x_data, @y_data)"), ("label", "@labels") ])

p = figure(title='Example Training Set', x_range=Range1d(-3, 3), tools = [hover],plot_width=480, plot_height=360)

p.scatter(x='x_data', y='y_data', size=6, source=source)
p.xaxis[0].axis_label = 'x'
p.yaxis[0].axis_label = 'y'

show(p)

### Defining our network

The goal of our network will be to take an $(x,y)$ point as input and to map that to the labels. If the network could compute $x^2 + y^2$, this network will have extracted the essential information from the input needed to classify the point. Although our network does have nonlinearities, they are not of this form. In this section we will explore the basic building blocks of a vanilla deep network.

#### The basic structure of a deep network

Our input is 2 dimensional, and will thus be implemented with a 2 neuron input layer. To put the input into the network, we will set the _activiation_ of one of the neurons in the input layer to $x_0$ and the other neuron to $y_0$ from an $(x_0,y_0)$ point of interest.

The activations of the input layer neurons will propagate through the network. In our specific case we will have one _hidden layer_ before the output. This hidden layer will have 3 neurons. Each input layer neuron will have connections to every hidden layer neuron. A single hidden layer neuron will thus take a linear sum of activations from the input layer, apply an offset, $b$, and then send the result of that linear transformation through a nonlinearity called the RELU function (to be described later). Thus, a single hidden unit will have 2 weights describing the strength with which each input activation affects itself. The activation of a hidden unit $i$, $a_i$, is thus given by:

$$a_i = relu( W_{ix} x_0 + W_{iy} y_0 + b_i)$$

Where $W_{ix}$ is the weight from the $x$ input neuron to the $i$-th hidden layer unit. In our case the input is 2-D and the hidden layer it feeds into is 3D, so the connectivity between input and hidden layer can be described by a 3 by 2 matrix, W. Then the activation of the hidden layer can be mathematically described as a function of the 2-D input vector, $\bf{s}$$:= [x_0,y_0]^T$:

$$a_i=relu(W_{ij}s_j+b_i)$$

Where $i$ is the ordinality of the hidden layer $\in [0,1,2]$ and $j$ is the ordinality of the input layer $\in [0,1]$. The entirety of training the neural network will be reduced to finding the values of all $W$s that solve the classifiction problem.

The hidden layer activations will then feed into an output layer, which will hopefully give us the correct labels for the input data. Since our labels are 2-D one hot vectors, the output layer will be made of 2 neurons. Our outout activations will be given by:

$$a_i = relu(W_{ij}s_j)$$

where in this case $i$ is over the output layer, and $j$ is over the hidden layer. For a final step, we will apply a _soft max_ function to the output layer so that we can interpret the output as a probability distribution. The softmax function is:

$$p_j = \frac{e^{a_j}}{\sum_i a_i}$$

where $p_j$ is interpreted to be the probability with which the $j$-th element of the output vector is 1, and the sum over i is over all output activations.





#### The nonlinearities in this network

The nonlinearity of our network is called a rectifying linear unit (RELU). The relu vanishes for all negative values, and keeps all non-negative values the same:

In [70]:
x_data = np.linspace(-10,10,100)     # define x valuee
y_data = tf.nn.relu(x_data).eval()   # apply the relu function via tensorflow

p = figure(title='The RELU function', x_range=Range1d(-10, 10),plot_width=400, plot_height=300)
p.line(x_data,y_data)
p.xaxis[0].axis_label = 'input'
p.yaxis[0].axis_label = 'output'
show(p)

This hardly seems the building block for $x^2 + y^2$. For now, just note that one can think of the relu as folding the input. We will soon visualize the activity of the network in order to see this origami in full.

### Setting up the network in TensorFlow

In this section of code we will implement the architecture of our network in TensorFlow. This takes the form of defining the input with a variable, _x_, and instantiating with random values the weight matrix from the input to the hidden layer, _W1_, the offset for the hidden layer units _offset1_, the weight matrix from the hidden layer to the output, _W2_, and the activations for the hidden layer, _a0_, and the activations for the output, _a1_. Because of the way tensorflow implements learning, we will not explicitly take the softmax at the moment.

In [75]:
# Instantiate the input
x = tf.placeholder(tf.float32, [None, 2]) #input is 2-D

# Instantiate the weight matrix (2 by 3) from the input (2-D) to the hidden layer (3-D)
W1 = tf.Variable(tf.random_uniform([2, 3]))

# Instantiate the hidden layer offsets
offset1 = tf.Variable(tf.random_uniform([3]))

# Instantiate the weight matrix (3 by 2) from the hidden layer (3-D) to the output (2-D)
W2 = tf.Variable(tf.random_uniform([3, 2])) 

# Define the equations for the hidden layer activities, as a function of the inputs x,
# the weights from the inputs to the hidden layer, W1, and the offsets of the hidden layer, offset1
a0 = tf.nn.relu(tf.add(tf.matmul(x, W1), offset1))

# Define the output activations, as a function of the activities of the hidden layer a0,
# and the weight matrix from the hidden to the output layer W2
a1 = tf.nn.relu(tf.matmul(a0, W2)) 

### Training the network

We will not explain the algorithm for training the network in detail here. Generally what happens is that the network is instantiated randomly, and then training examples are put the the network, output labels are computed and compared to the correct labels, defining an _error_, and then that error is back-propagated through the network in order to change the weights proportional to the weights contribution to the output. The function used to compute the error is the _cross entropy_. In the following code, we will set up the equations to compute the error, and set the parameters for the optimization.

In [76]:
# define the output of the network as the output activations.
# we will apply softmax directly in the calculation of the error
y = a1

# define the vector for the true output
y_ = tf.placeholder(tf.float32, [None, 2]) #true output

# Here we will define the error (and apply softmax)
# Tensorflow has a function which both applys the softmax and 
# applies the cross entropy in on function.
#
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))


In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

# Train
for i in range(1000):
    batch_xs, batch_ys = makeTraining()
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [ ]:
# Get network from tensorflow

m1 = numpy.transpose(W1.eval())
b = numpy.transpose(offset1.eval())
m2 = numpy.transpose(W2.eval())

In [ ]:
# create inputs
grid = 15

x = numpy.linspace(-2, 2, grid)
y = numpy.linspace(-2, 2, grid)
grid_size = x[1]-x[0]
circle_mesh_x, circle_mesh_y = numpy.meshgrid(x, y)
Z = np.zeros((grid,grid))
for i in range(grid):
    for j in range(grid):
        Z[i,j] = (circle_mesh_x[i,j]+0.5*grid_size)**2 + (circle_mesh_y+0.5*grid_size)[i,j]**2

N = Z/Z.max() # for colors

mesh = [circle_mesh_x.reshape(grid**2), circle_mesh_y.reshape(grid**2)]
        

In [ ]:
# run the inputs through the first layer

output_mesh = [mesh[0], mesh[1], np.zeros((grid,grid))]
#output_mesh = tf.nn.relu(numpy.matmul(m1,mesh) + numpy.transpose(numpy.matlib.repmat(b, 100, 1))).eval() # first layer


# Input Layer

In [ ]:
# initialize plot

%matplotlib notebook
fig = plt.figure()

In [ ]:
# run plot
plt.pcolor(circle_mesh_x, circle_mesh_y, N, edgecolors='none')
plt.show()

# Layer 1

In [ ]:
# run the inputs through the first layer
# output_mesh = tf.nn.relu(numpy.matmul(m1,mesh) + numpy.transpose(numpy.matlib.repmat(b, 100, 1))).eval() # first layer

output_mesh0 = output_mesh
output_mesh1 = numpy.matmul(m1,mesh)
output_mesh2 = output_mesh1 + numpy.transpose(numpy.matlib.repmat(b, grid**2, 1))
output_mesh3 = tf.nn.relu(output_mesh2).eval()



In [ ]:
# initialize plot

%matplotlib notebook
fig2 = plt.figure(figsize=plt.figaspect(0.7))


In [ ]:
# run plot
ax = fig2.add_subplot(221, projection='3d')
surf = ax.plot_surface(output_mesh0[0].reshape(grid,grid),output_mesh0[1].reshape(grid,grid), output_mesh0[2].reshape(grid,grid), rstride=1, cstride=1,facecolors=cm.jet(N),linewidth=0, antialiased=False, shade=False)


ax1 = fig2.add_subplot(222, sharex=ax, sharey=ax,sharez=ax, projection='3d')
surf = ax1.plot_surface(output_mesh1[0].reshape(grid,grid),output_mesh1[1].reshape(grid,grid), output_mesh1[2].reshape(grid,grid), rstride=1, cstride=1,facecolors=cm.jet(N),linewidth=0, antialiased=False, shade=False)

ax2 = fig2.add_subplot(223, sharex=ax, sharey=ax,sharez=ax, projection='3d')
surf = ax2.plot_surface(output_mesh2[0].reshape(grid,grid),output_mesh2[1].reshape(grid,grid), output_mesh2[2].reshape(grid,grid), rstride=1, cstride=1,facecolors=cm.jet(N),linewidth=0, antialiased=False, shade=False)

ax3 = fig2.add_subplot(224, sharex=ax, sharey=ax,sharez=ax, projection='3d')
surf = ax3.plot_surface(output_mesh3[0].reshape(grid,grid),output_mesh3[1].reshape(grid,grid), output_mesh3[2].reshape(grid,grid), rstride=1, cstride=1,facecolors=cm.jet(N),linewidth=0, antialiased=False, shade=False)

ax_limits = 20
ax.set_xlim([-ax_limits,ax_limits])
ax.set_ylim([-ax_limits,ax_limits])
ax.set_zlim([-ax_limits,ax_limits])

plt.show()

# Output Layer

In [ ]:
output_mesh4 = numpy.matmul(m2,output_mesh3)
#output_mesh5 = [tf.nn.softmax([output_mesh4[0][i],output_mesh4[1][i]]).eval()[0] for i in range(grid**2)]
output_mesh5 = [[np.exp(output_mesh4[0][i])/(np.exp(output_mesh4[0][i]) + np.exp(output_mesh4[1][i])) for i in range(grid**2)], [np.exp(output_mesh4[1][i])/(np.exp(output_mesh4[0][i]) + np.exp(output_mesh4[1][i])) for i in range(grid**2)]]
output_mesh5 = np.array(output_mesh5)

In [ ]:
# initialize plot

%matplotlib notebook
fig3 = plt.figure()

In [ ]:
# run plot
ax4 = fig3.add_subplot(121)
surf = ax4.pcolor(output_mesh4[0].reshape(grid, grid), output_mesh4[1].reshape(grid, grid),N)#, N, edgecolors='none')

ax5 = fig3.add_subplot(122)
surf = ax5.pcolor(output_mesh5[0].reshape(grid, grid), output_mesh5[1].reshape(grid, grid), N, edgecolors='none')

ax4.set_xlim([-65,65])
ax4.set_ylim([-65,65])

plt.show()

In [ ]:
# initialize plot

%matplotlib notebook
fig4 = plt.figure()

In [ ]:
ax = fig4.add_subplot(111)
surf = ax.pcolor(circle_mesh_x, circle_mesh_y,output_mesh5[0].reshape(grid,grid))#, N, edgecolors='none')
cbar = fig.colorbar(surf, ticks=[-1, 0, 1])